
discord remote control?

export TOKENPATH=/Users/briancullinan/.credentials/discord-bot6.txt && export DEFAULT_APPLICATION=1335491680630997034 && npm run import -- "discord remote control"




In [ ]:

const {
  registerCommand, getCommands, deleteCommand, updateCommand
} = importer.import('discord api')
const {interactionsCommands} = importer.import('discord gateway')
const {discordExpress, closeExpress} = importer.import('discord remote proxy server')

const ALL_COMMANDS = [
  'remote',
]

// bot commands using new API, same names as above but lower-case
async function activityCommands(guildId = null) {
  var cmd, cmdDef
  var commandResult = await getCommands(guildId)
  var commands = commandResult.map(command => command.name)

  interactionsCommands['startActivity'] = discordExpress
  interactionsCommands['endActivity'] = closeExpress

  cmdDef = {
    'name': 'remote',
    'description': 'Launch the remote',
    'type': 4,
    'handler': 2,
  }
  if(!commands.includes('remote')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'remote')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }

  var toRemove = commandResult.filter(c => !ALL_COMMANDS.includes(c.name))
  for(var i = 0; i < toRemove.length; i++) {
    await deleteCommand(toRemove[i].id, guildId)
  }

  return await getCommands()
}

module.exports = activityCommands





discord remote proxy server?


In [ ]:
const {doClick, getMonitor, getToken} = importer.import('express automation routes')

let instances = {}
const express = require('express')
const cors = require('cors')
const app = express()

app.use(cors({
  origin: '*' 
}))
app.use(express.json());
app.post('/click', doClick);
app.get('/monitors/*', getMonitor);
app.post('/api/token', getToken);
//app.get('*', handleRoute);

let server

async function discordExpress(activity) {
  if(!express) {
    return false
  }
  
  process.env['NODE_TLS_REJECT_UNAUTHORIZED'] = 0;

  if(activity.participants.length == 0 && server) {
    server.close()
    server = null
  }

  // TODO: check if user was removed
  if(typeof instances[activity['instance_id']] == 'undefined') {
    instances[activity['instance_id']] = []
  }
  instances[activity['instance_id']] = instances[activity['instance_id']].filter(i => activity.participants.includes(i[0]))

  if(activity.length > 1 || server) {
    return
  }

  return (server = await app.listen(3000, () => {
    console.log('Listening on: ', 3000)
  }))
}

function closeExpress(activity) {
  if(activity) {
    if(typeof instances[activity['instance_id']] != 'undefined') {
      instances[activity['instance_id']] = instances[activity['instance_id']].filter(i => activity.participants.includes(i[0]))
    }
    return
  }
  if(server) {
    server.close()
    server = null
  }
}

module.exports = {
  discordExpress,
  closeExpress,
}





express automation routes?


In [ ]:
const {request} = require('gaxios')
const {DEFAULT_API, SECRET, TOKEN, DEFAULT_APPLICATION} = importer.import('discord configuration')
const screenshotMonitor = importer.import('node screenshots')
const qs = require('qs');

async function getToken(req, res, next) {
  //console.log(req.body)
  let result = await request({
    url: DEFAULT_API + 'oauth2/token',
    method: 'POST',
    headers: {
      'Content-Type': 'application/x-www-form-urlencoded',
      //'Authorization': 'Bot ' + TOKEN
    },
    data: qs.stringify({
      client_id: DEFAULT_APPLICATION,
      client_secret: SECRET,
      grant_type: 'authorization_code',
			code: req.body.code,
		})
  })
  //console.log(result)
  res.setHeader('Content-Type', 'application/json')
  return res.send(JSON.stringify({access_token: result.data.access_token}))
}

async function getMonitor(req, res, next) {
  let index = parseInt(req.path.replace(/^\/|\/$/gi, '').split('/')[1])
  if(Number.isNaN(index)) {
    index = 0
  }
  let screenshot = await screenshotMonitor(index)
  if(!screenshot) {
    return next()
  }
  res.setHeader('Content-Type', 'image/png')
  return res.send(screenshot)
}

async function doClick(req, res, next) {
  try {
    const { Monitor } = require("node-screenshots");
    const MouseController = require('mouse-controller');
    const mc = new MouseController();
    let monitors = Monitor.all()
    mc.move(req.body.x * monitors[0].width * 3, req.body.y * monitors[0].height)
  } catch (e) {
    console.log(e)
  }
  return res.send('')
}

module.exports = {
  doClick,
  getMonitor,
  getToken,
}
